This Lab contains the following tasks:

1: Mark all launch sites on a map

2: Mark the success/failed launches for each site on the map

3: Calculate the distances between a launch site to its proximities


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [3]:
#The following dataset with the name spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site.
#from js import fetch
import io
import requests

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
s=requests.get(URL).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [4]:
#Now, We can take a look at what are the coordinates for each site.(`Launch Site`, `Lat(Latitude)`, `Long(Longitude)`)

spacex_df = df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [7]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [8]:
# Iterate through each launch site and add a Circle object to the map
for index, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    lat = row['Lat']
    long = row['Long']
    
    # Create a Circle object with the launch site coordinates
    circle = folium.Circle(
        location=[lat, long],
        radius=1000,
        color='#d35400',
        fill=True,
        fill_color='#3186cc'
    ).add_child(folium.Popup(launch_site))

    #crate an icon as text label for each launch site
    marker = folium.map.Marker(
        [lat, long],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_site,
            )
        )
    
    # Add the Circle object to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map

In [9]:
# Define a function to assign colors based on launch success
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# Iterate through each launch and add a CircleMarker to the map
for index, row in spacex_df.iterrows():
    launch_site = row['Launch Site']
    lat = row['Lat']
    long = row['Long']
    outcome = row['class']
    
    # Assign color based on launch outcome
    marker_color = assign_marker_color(outcome)
    
    # Create a CircleMarker object with the launch site coordinates
    circle_marker = folium.CircleMarker(
        location=[lat, long],
        radius=5,
        color=marker_color,
        fill=True,
        fill_color=marker_color,
        fill_opacity=0.6
    ).add_child(folium.Popup(f"{launch_site} - {'Success' if outcome == 1 else 'Failure'}"))

    # Add the CircleMarker object to the map
    site_map.add_child(circle_marker)

# Display the map
site_map

In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [11]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [12]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distanc

In [13]:
from math import radians, cos, sin, sqrt, atan2

# Function to calculate the distance between two points using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Calculate distances from each launch site to all other launch sites
distances = []
for i, row1 in launch_sites_df.iterrows():
    for j, row2 in launch_sites_df.iterrows():
        if i != j:
            distance = haversine(row1['Lat'], row1['Long'], row2['Lat'], row2['Long'])
            distances.append({
                'From': row1['Launch Site'],
                'To': row2['Launch Site'],
                'Distance (km)': distance
            })

# Convert distances to a DataFrame for better visualization
distances_df = pd.DataFrame(distances)
distances_df

,From,To,Distance (km)
0,CCAFS LC-40,CCAFS SLC-40,0.112488
1,CCAFS LC-40,KSC LC-39A,6.899304
2,CCAFS LC-40,VAFB SLC-4E,3825.840279
3,CCAFS SLC-40,CCAFS LC-40,0.112488
4,CCAFS SLC-40,KSC LC-39A,6.934099
5,CCAFS SLC-40,VAFB SLC-4E,3825.854482
6,KSC LC-39A,CCAFS LC-40,6.899304
7,KSC LC-39A,CCAFS SLC-40,6.934099
8,KSC LC-39A,VAFB SLC-4E,3819.052794
9,VAFB SLC-4E,CCAFS LC-40,3825.840279


In [14]:
# Define the coordinates for the selected coastline point
coastline_point = [28.56319718, -80.57682003]  # Example coordinates

# Define the coordinates for the launch site
launch_site_coordinates = [lat, long]

# Create a PolyLine object with the launch site and coastline point coordinates
polyline = folium.PolyLine(
    locations=[launch_site_coordinates, coastline_point],
    color='blue',
    weight=2,
    opacity=1
)

# Add the PolyLine object to the map
site_map.add_child(polyline)

# Display the map with the PolyLine
site_map

In [15]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 5);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 5);}",
)

site_map.add_child(mouse_position)

# Display the map with Mouse Position
site_map

In [16]:
# Choose a launch site to zoom in
zoom_launch_site = 'CCAFS LC-40'
zoom_site = launch_sites_df[launch_sites_df['Launch Site'] == zoom_launch_site].iloc[0]
zoom_coordinate = [zoom_site['Lat'], zoom_site['Long']]

# Create a new folium map centered at the chosen launch site with a higher zoom level
zoom_map = folium.Map(location=zoom_coordinate, zoom_start=15)

# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
mouse_position_zoom = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=5,
    prefix='Lat:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 5);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 5);}",
)

zoom_map.add_child(mouse_position_zoom)

# Display the zoomed-in map
zoom_map